# 💡 Investment FAQ Chatbot with Vector Search  

This project implements an **AI-powered FAQ chatbot** for an online investment platform. The chatbot uses **vector embeddings** and **semantic search** to find the most relevant answers to user queries based on a stored knowledge base of frequently asked questions.  

## ✨ Key Features  
- **SQLite Database with Vector Search**: Stores FAQ data and performs similarity-based retrieval using embeddings.  
- **Text Embeddings**: Converts questions into numerical vectors for efficient comparison.  
- **Retrieval-Augmented Generation (RAG)**: Finds relevant information before answering user queries.  
- **LLM-powered Responses**: Uses `Groq`’s `llama-3.3-70b-versatile` model to generate precise and context-aware answers.  

## 📌 How It Works  
1. **Store FAQs in a Database**: The knowledge base is stored in an SQLite table.  
2. **Generate & Store Embeddings**: Each FAQ is embedded into a vector representation.  
3. **Search for Relevant Answers**: When a user asks a question, the system finds the most similar stored questions.  
4. **Generate AI Response**: A structured prompt is sent to the language model, ensuring that responses are based only on available data.  

## 🚀 Use Case  
This chatbot is ideal for **automated customer support** in financial services, providing **accurate and instant answers** to investment-related queries.  

Run the following cells to set up the environment and start querying the chatbot! 🎯


###1. Check if it is possible to use GPU to accelerate task:

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


###2. Install the relevant packages:

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq fastembed==0.3.4 --progress-bar off
!pip install -qqq sqlite-vec==0.1.1 --progress-bar off
!pip install -qqq groq --progress-bar off
!pip install -qqq --upgrade httpx
!pip install --upgrade groq

###3. Import relevant libraries:
SQLite library: handle database operations

dedent: remove leading whitespace from multi-line strings

List: type hinting

sqlite_vec: to enable vector search in SQLite

fastembed's TextEmbedding: text vectorization

userdata: retrieve user-specific data (API keys)

groq: interacting with the Groq API

Groq client: for API requests

Import ChatCompletionMessage type

serialize_float32: serialization function for float32 embeddings

In [3]:
import sqlite3
from textwrap import dedent
from typing import List

import sqlite_vec
from fastembed import TextEmbedding
from google.colab import userdata
import groq
from groq import Groq
from groq.types.chat import ChatCompletionMessage
from sqlite_vec import serialize_float32

###4. Connect to SQLite database named 'faq.sqlite3'
Enable loading of SQLite extensions

Load the sqlite_vec extension for vector operations

Disable further loading of extensions for security

###5. Set up the LLM using Groq
Retrieve Groq API key from Colab's user data

Initialize Groq API client with the retrieved API key

Define model settings for LLM, model name, 0 for deterministic response, max token limit

In [5]:
db = sqlite3.connect("faq.sqlite3")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

groq_api_key = userdata.get('GROQ_API_KEY')

client = Groq(api_key=groq_api_key)
MODEL = "llama-3.3-70b-versatile"
TEMPERATURE = 0
MAX_TOKENS = 4096



###6. Load SQL extension in Google Colab Notebook
Connect to SQLite database within Jupyter

In [6]:
%load_ext sql
%sql sqlite:///faq.sqlite3

###7. Define a list of FAQs (knowledge base)

In [7]:
KNOWLEDGE_BASE = [
    {
        "question": "What types of investment accounts do you offer?",
        "answer": "We offer a range of investment accounts including individual brokerage accounts, retirement accounts (IRA, Roth IRA), and managed portfolios. Each account type is designed to meet different financial goals and risk tolerances.",
    },
    {
        "question": "What are your fees for trading stocks?",
        "answer": "We offer commission-free trading for stocks and ETFs. However, options trading and other advanced trading features may have associated fees. Please refer to our fee schedule for full details.",
    },
    {
        "question": "Do you provide financial advisory services?",
        "answer": "Yes, we offer both automated and human advisory services. Our robo-advisory platform provides algorithm-driven investment strategies, while our certified financial advisors offer personalized financial planning.",
    },
    {
        "question": "How do I open an account?",
        "answer": "You can open an account online by providing your personal details, financial information, and identity verification documents. The process typically takes a few minutes, and you can start investing once your account is approved.",
    },
    {
        "question": "Is my money insured?",
        "answer": "Yes, cash balances in your investment account are insured by the FDIC up to applicable limits when held in a partner bank. Securities are protected by SIPC up to $500,000, including $250,000 for cash claims.",
    },
    {
        "question": "What investment options do you offer?",
        "answer": "We provide a variety of investment options including stocks, bonds, ETFs, mutual funds, and alternative assets. Our platform supports both self-directed investing and automated portfolio management.",
    },
    {
        "question": "How can I deposit or withdraw funds?",
        "answer": "You can deposit funds via bank transfer, wire transfer, or mobile check deposit. Withdrawals can be requested through bank transfer or wire transfer, with processing times varying depending on the method chosen.",
    },
    {
        "question": "Do you offer retirement planning services?",
        "answer": "Yes, we provide retirement planning tools and expert guidance to help you build a solid financial future. Our advisors can assist with IRA rollovers, 401(k) transfers, and retirement income strategies.",
    },
    {
        "question": "How do I track my investments?",
        "answer": "You can track your portfolio performance through our mobile app or website. We provide real-time updates, detailed analytics, and personalized insights to help you monitor your investments.",
    },
    {
        "question": "What security measures do you have in place?",
        "answer": "We use state-of-the-art security measures including two-factor authentication, encryption, and fraud monitoring to protect your account and personal information.",
    },
    {
        "question": "Do you offer tax-advantaged investment options?",
        "answer": "Yes, we offer tax-advantaged accounts such as IRAs and 529 college savings plans. Additionally, we provide tax-loss harvesting strategies to help minimize taxable gains.",
    },
    {
        "question": "Can I invest in cryptocurrencies?",
        "answer": "Yes, we provide access to cryptocurrency investments through select funds, ETFs, and direct crypto trading on our platform. Please note that cryptocurrency investments come with higher risks and volatility.",
    },
]


###8. Create 'documents' table to store FAQs in SQLite

In [8]:
db.execute(
    """
    CREATE TABLE documents(
        id INTEGER PRIMARY KEY,
        question TEXT,
        answer TEXT
    );
"""
)

###9. Insert knowledge base entries into the 'documents' table

In [9]:

with db:
    for i, doc in enumerate(KNOWLEDGE_BASE):
        db.execute(
            "INSERT INTO documents(id, question, answer) VALUES(?, ?, ?)",
            [i, doc["question"], doc["answer"]],
        )

###10. Initialize text embedding model

In [10]:
embedding_model = TextEmbedding()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

###11. Generate embedding for the first FAQ question (as demo)
Get the length and shape of the embedding of the demo

In [11]:
document_embeddings = list(embedding_model.embed(KNOWLEDGE_BASE[0]["question"]))
len(document_embeddings), document_embeddings[0].shape

(1, (384,))

###12. Create a virtual table named 'document_embeddings' in the SQLite database.

This table is used to store vector embeddings of documents, allowing for efficient similarity searches.

i  Create a virtual table using the vec0 module for vector search

ii  Define 'id' as the primary key (unique identifier for each document)

iii  Define 'embedding' column to store vector embeddings

  The size of the FLOAT array is dynamically set based on the length of the generated embeddings

  e.g., if each embedding has 768 dimensions, the column will store a FLOAT[768] vector




In [12]:
db.execute(
    f"""
        CREATE VIRTUAL TABLE document_embeddings USING vec0(
          id INTEGER PRIMARY KEY,
          embedding FLOAT[{document_embeddings[0].shape[0]}]
        );
    """
)

###13.  Fetch all documents from the 'documents' table and store them in a list

Fetch all documents

Combine question and answer into a single string

Print the first document

Generate embeddings for all documents



In [13]:
with db:
    document_rows = db.execute("SELECT id, question, answer FROM documents").fetchall()
    documents = [f"{row[1]}\n{row[2]}" for row in document_rows]
    print(documents[0])
    document_embeddings = list(embedding_model.embed(documents))

    #Insert document embeddings into 'document_embeddings' table
    for (id, _, _), embedding in zip(document_rows, document_embeddings):
        db.execute(
            "INSERT INTO document_embeddings(id, embedding) VALUES (?, ?)", #Insert SQL query
            [id, serialize_float32(embedding)],  # Convert embedding to float32 and insert
        )

What types of investment accounts do you offer?
We offer a range of investment accounts including individual brokerage accounts, retirement accounts (IRA, Roth IRA), and managed portfolios. Each account type is designed to meet different financial goals and risk tolerances.


###14. Query SQLite metadata to view database structure

In [14]:
%%sql
SELECT
    name,
    type,
    sql
FROM
    sqlite_master
WHERE
    type IN ('table', 'index', 'view', 'trigger')
ORDER BY
    type,
    name;

 * sqlite:///faq.sqlite3
Done.


KeyError: 'DEFAULT'

###15. Define a query for testing
Generate embedding for query

Extract first embedding

In [15]:
query = "How can I check my investment, is it safe?"
query_embeddings = list(embedding_model.embed([query]))
query_embedding = query_embeddings[0]

###16. Perform a vector similarity search in the database to find the top 3 most relevant questions.

####Retrieve the:

unique ID of the matched document

The similarity score (distance metric) between query and stored embeddings

The question from the original document

The corresponding answer from the original document


Then:

Join with the 'documents' table to get question-answer pairs

Perform vector similarity search using the query embedding

Retrieve the top 3 closest matches based on similarity

Sort the results by similarity (lower distance means more relevant)

Lastly:

Convert query embedding to float32 format for compatibility with the database


Execute the query and fetch all results



In [16]:
results = db.execute(
    """
      SELECT
        document_embeddings.id,
        distance,
        question,
        answer
      FROM document_embeddings
      LEFT JOIN documents ON documents.id = document_embeddings.id
      WHERE embedding MATCH ?
        AND k = 3
      ORDER BY distance
    """,
    [serialize_float32(query_embedding)],
).fetchall()

In [17]:
results

[(8,
  0.7390538454055786,
  'How do I track my investments?',
  'You can track your portfolio performance through our mobile app or website. We provide real-time updates, detailed analytics, and personalized insights to help you monitor your investments.'),
 (4,
  0.756416380405426,
  'Is my money insured?',
  'Yes, cash balances in your investment account are insured by the FDIC up to applicable limits when held in a partner bank. Securities are protected by SIPC up to $500,000, including $250,000 for cash claims.'),
 (3,
  0.8273014426231384,
  'How do I open an account?',
  'You can open an account online by providing your personal details, financial information, and identity verification documents. The process typically takes a few minutes, and you can start investing once your account is approved.')]

###17. Define a function to retrieve context based on user query

In [18]:
def retrieve_context(
    query: str, k: int = 3, embedding_model: TextEmbedding = embedding_model
) -> str:                               # Function takes a query, number of results (k), and an embedding model
    query_embedding = list(embedding_model.embed([query]))[0]
    results = db.execute(
        """
        SELECT
            document_embeddings.id,
            distance,
            question,
            answer
        FROM document_embeddings
        LEFT JOIN documents ON documents.id = document_embeddings.id
        WHERE embedding MATCH ?
            AND k = ?
        ORDER BY distance
        """,
        [serialize_float32(query_embedding), k],
    ).fetchall()

     # Format the results as a string, separating each question-answer pair with "---"
    return "\n---\n".join([f"{item[2]}\n{item[3]}" for item in results])

###18. Test retrieval function

In [19]:
query = "How can I check my investment, is it safe?"
print(retrieve_context(query))

How do I track my investments?
You can track your portfolio performance through our mobile app or website. We provide real-time updates, detailed analytics, and personalized insights to help you monitor your investments.
---
Is my money insured?
Yes, cash balances in your investment account are insured by the FDIC up to applicable limits when held in a partner bank. Securities are protected by SIPC up to $500,000, including $250,000 for cash claims.
---
How do I open an account?
You can open an account online by providing your personal details, financial information, and identity verification documents. The process typically takes a few minutes, and you can start investing once your account is approved.


###19. Define system prompt for chatbot (for LLM)

In [20]:
SYSTEM_PROMPT = """ You're a senior customer support agent for an online investment platform.
You're always helpful and answer customer questions only based on the provided information.
If you don't know the answer – just reply with an excuse that you don't know.
Keep your answers brief and to the point. Be kind and respectful.

Use the provided context for your answers. The most relevant information is at the top.
Each piece of information is separated by ---.
"""

###20. Function to create a user prompt, with the information from the knowledge base as context

Removes common leading whitespace for better formatting

Calls retrieve_context() to get relevant question-answer pairs from the database

Includes the original user query at the end

In [21]:
def create_user_prompt(query: str) -> str:
    return dedent(
        f"""
Use the following information:

```
{retrieve_context(query)}
```

to answer the question:
{query}
    """
    )

In [22]:
print(create_user_prompt(query))


Use the following information:

```
How do I track my investments?
You can track your portfolio performance through our mobile app or website. We provide real-time updates, detailed analytics, and personalized insights to help you monitor your investments.
---
Is my money insured?
Yes, cash balances in your investment account are insured by the FDIC up to applicable limits when held in a partner bank. Securities are protected by SIPC up to $500,000, including $250,000 for cash claims.
---
How do I open an account?
You can open an account online by providing your personal details, financial information, and identity verification documents. The process typically takes a few minutes, and you can start investing once your account is approved.
```

to answer the question:
How can I check my investment, is it safe?



###21. Store user prompt and Define the initial message structure for LLM API

In [23]:
user_prompt = create_user_prompt(query)

messages = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT,
    },
]

###22. Function to call the AI model for generating responses

In [24]:
def call_model(query: str, messages: List) -> ChatCompletionMessage:
  #Append user query to message history
    messages.append(
        {
            "role": "user",
            "content": create_user_prompt(query),
        },
    )
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,   # Pass conversation history
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
    )
    message = response.choices[0].message   # Extract response
    messages.append(message)    # Store response in conversation history
    return message

###23. Test model with query

In [25]:
query = "How can I check my investment, is it safe?"
message = call_model(query, messages)
print(message)

ChatCompletionMessage(content='You can track your investments through our mobile app or website, which provides real-time updates. As for safety, your cash balances are insured by the FDIC and securities are protected by SIPC up to $500,000.', role='assistant', function_call=None, reasoning=None, tool_calls=None)


In [26]:
print(message.content)

You can track your investments through our mobile app or website, which provides real-time updates. As for safety, your cash balances are insured by the FDIC and securities are protected by SIPC up to $500,000.


####In Comparison:
"question": "How do I track my investments?",
        
"answer": "You can track your portfolio performance through our mobile app or website. We provide real-time updates, detailed analytics, and personalized insights to help you monitor your investments.",

"question": "Is my money insured?",

"answer": "Yes, cash balances in your investment account are insured by the FDIC up to applicable limits when held in a partner bank. Securities are protected by SIPC up to $500,000, including $250,000 for cash claims.",

#### Another Test sample

In [27]:
query = "Is there any free of tax offering and can I buying crypto?"
message = call_model(query, messages)
print(message.content)

We offer tax-advantaged accounts such as IRAs and 529 college savings plans, which can help minimize taxable gains. Additionally, yes, you can invest in cryptocurrencies through our platform, but please note that they come with higher risks.


####In Comparison:

"question": "Do you offer tax-advantaged investment options?",

"answer": "Yes, we offer tax-advantaged accounts such as IRAs and 529 college savings plans. Additionally, we provide tax-loss harvesting strategies to help minimize taxable gains."

"question": "Can I invest in cryptocurrencies?",

"answer": "Yes, we provide access to cryptocurrency investments through select funds, ETFs, and direct crypto trading on our platform. Please note that cryptocurrency investments come with higher risks and volatility."

#### Test sample where the user ask irrelevant questions:

In [28]:
query = "How long did it take me to complete this project?"
message = call_model(query, messages)
print(message.content)

I don't know. The information provided is about opening an account, tracking investments, and depositing/withdrawing funds, but it doesn't mention a project or its completion time.
